### Introduction
In this notebook, we will test out pulling out Microsoft's [Phi3:14b](https://ollama.com/library/phi3) and serving with [Ollama](https://ollama.com/) in SAP AI Core. You can also run LLaMa 3, Phi3, Mistral, Mixtral, LLaVa, and other [supported models in Ollama](https://ollama.com/library). <br/><br/>

Please refer to this [blog post](https://community.sap.com/t5/artificial-intelligence-and-machine-learning-blogs/bring-open-source-llms-into-sap-ai-core-with-ollama/ba-p/13659769) about Bring Open-Source LLMs into SAP AI Core with Ollama for more details.

### Prerequisites
Before running this notebook, please assure you have performed the [Prerequisites](../../README.md) and [01-deployment.ipynb](01-deployment.ipynb). As a result, a deployment of Ollama scenario is running in SAP AI Core. <br/><br/>

If the configuration and deployment are created through SAP AI Launchpad, please manually update the configuration_id and deployment_id in [env.json](env.json)
```json
{
    "configuration_id": "<YOUR_CONFIGURATION_ID_OF_OLLAMA_SCENARIO>",
    "deployment_id": "<YOUR_DEPLOYMENT_ID_BASED_ON_CONFIG_ABOVE>"
}
```
 
### The high-level flow:
- Load configurations info
- Connect to SAP AI Core via SDK
- Check the status and logs of the deployment
- Pull model from ollama model repository through API
- Inference the model with OpenAI-compatible chat completion API


#### 1.Load config info 
- resource_group loaded from [config.json](../config.json)
- deployment_id(created in 01-deployment.ipynb) loaded [env.json](env.json)

In [27]:
import requests, json
from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client

In [28]:
# Please replace the configurations below.
# config_id: The target configuration to create the deployment. Please create the configuration first.
with open("../config.json") as f:
    config = json.load(f)

with open("./env.json") as f:
    env = json.load(f)

deployment_id = env["deployment_id"]
resource_group = config.get("resource_group", "default")
print("deployment id: ", deployment_id, " resource group: ", resource_group)

deployment id:  df240ccfb2d899b0  resource group:  oss-llm


#### 2.Initiate connection to SAP AI Core 

In [29]:
aic_sk = config["ai_core_service_key"]
base_url = aic_sk["serviceurls"]["AI_API_URL"] + "/v2/lm"
ai_api_client = AIAPIV2Client(
    base_url= base_url,
    auth_url=aic_sk["url"] + "/oauth/token",
    client_id=aic_sk['clientid'],
    client_secret=aic_sk['clientsecret'],
    resource_group=resource_group)


In [30]:
token = ai_api_client.rest_client.get_token()
headers = {
        "Authorization": token,
        'ai-resource-group': resource_group,
        "Content-Type": "application/json"}


#### 3.Check the deployment status 

In [31]:
# Check deployment status before inference request
deployment_url = f"{base_url}/deployments/{deployment_id}"
response = requests.get(url=deployment_url, headers=headers)
resp = response.json()    
status = resp['status']

deployment_log_url = f"{base_url}/deployments/{deployment_id}/logs"
if status == "RUNNING":
        print(f"Deployment-{deployment_id} is running. Ready for inference request")
else:
        print(f"Deployment-{deployment_id} status: {status}. Not yet ready for inference request")
        #retrieve deployment logs
        #{{apiurl}}/v2/lm/deployments/{{deploymentid}}/logs.

        response = requests.get(deployment_log_url, headers=headers)
        print('Deployment Logs:\n', response.text)


Deployment-df240ccfb2d899b0 is running. Ready for inference request


#### 4.Pull the model into Ollama 

In [32]:
model = "phi3:14b"
#model = "llama3:8b"
#model = "mistral:latest"
#model = "phi3:latest" 
#model = "mistral:7b-instruct-q5_K_M"
#model = "mixtral:8x7b-instruct-v0.1-q4_0" #Important: please resource plan to infer.l in byom-oss-llm-templates/ollama-template.yaml

deployment = ai_api_client.deployment.get(deployment_id)
inference_base_url = f"{deployment.deployment_url}/v1"
openai_base_url = deployment.deployment_url

In [ ]:
# pull the model from ollama model repository
endpoint = f"{inference_base_url}/api/pull"
print(endpoint)

#let's pull the mistral model from ollama
json_data = {  "name": model}

response = requests.post(endpoint, headers=headers, json=json_data)
print('Result:', response.text)

Next, let's list the model and check if the target model is listed. 

In [34]:
# Check the model list 
endpoint = f"{inference_base_url}/api/tags"
print(endpoint)

response = requests.get(endpoint, headers=headers)
print('Result:', response.text)

https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/df240ccfb2d899b0/v1/api/tags
Result: {"models":[{"name":"phi3:14b","model":"phi3:14b","modified_at":"2024-05-23T09:50:06.013426087Z","size":7897126241,"digest":"1e67dff39209b792d22a20f30ebabe679c64db83de91544693c4915b57e475aa","details":{"parent_model":"","format":"gguf","family":"phi3","families":["phi3"],"parameter_size":"14.0B","quantization_level":"F16"},"expires_at":"0001-01-01T00:00:00Z"},{"name":"llama3:latest","model":"llama3:latest","modified_at":"2024-05-23T02:47:43.160643007Z","size":4661224676,"digest":"365c0bd3c000a25d28ddbf732fe1c6add414de7275464c4e4d1c3b5fcb5d8ad1","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8.0B","quantization_level":"Q4_0"},"expires_at":"0001-01-01T00:00:00Z"},{"name":"llama3:8b","model":"llama3:8b","modified_at":"2024-05-23T02:19:25.163506171Z","size":4661224676,"digest":"365c0bd3c000a25d28ddbf732fe1c6add414de7275

#### 5.Inference completion and chat completion APIs

In [35]:
completion_api_endpoint = f"{inference_base_url}/api/generate"
chat_api_endpoint = f"{inference_base_url}/api/chat"
openai_chat_api_endpoint = f"{openai_base_url}/v1/chat/completions"

##### 5.1 Sample#1: Test Ollama's Completion API
Let's test it with a general Q&A.

In [21]:
#test ollama's completion api
json_data = {
  "model": model,
  "prompt": "What color is the sky at different times of the day? Respond in JSON",
  "format": "json", #JSON mode
  "stream": False   #Streaming or not
}

response = requests.post(url=completion_api_endpoint, headers=headers, json=json_data)
print('Result:', response.text)

Result: {"model":"phi3:14b","created_at":"2024-05-23T09:56:47.107018179Z","response":"{\n  \"Sunrise\": \"Pinkish to orange\",\n  \"Morning\": \"Light blue\",\n  \"Afternoon\": \"Blue\",\n  \"Evening\": \"Orange and red hues, eventually transitioning to darker shades of blue or black at night\"\n}","done":true,"done_reason":"stop","context":[32010,13,5618,2927,338,278,14744,472,1422,3064,310,278,2462,29973,2538,2818,297,4663,32007,13,32001,13,29912,13,29871,376,29903,348,29878,895,1115,376,29925,682,728,304,24841,613,13,29871,376,28581,1076,1115,376,20769,7254,613,13,29871,376,29909,29888,16691,1115,376,21319,613,13,29871,376,29923,854,292,1115,376,29949,3881,322,2654,298,1041,29892,10201,9558,292,304,6501,261,528,3076,310,7254,470,4628,472,4646,29908,13,29913,32007],"total_duration":4143650900,"load_duration":1225039,"prompt_eval_count":20,"prompt_eval_duration":354871000,"eval_count":69,"eval_duration":3740907000}


##### 5.2 Test Ollama's Chat Completion API

Now let's test Ollama's [OpenAI compatible API for Chat Completion](https://github.com/ollama/ollama/blob/main/docs/openai.md), which is the exact API interface of Chat Completion of GPT-3.5/4 in SAP Generative AI Hub. 
##### Sample#2: Write a haiku about Ollama in AI Core
Let's test its chat completion API

In [22]:
#let's test ollama openai-compatible chat completion api by writing a haiku
sys_msg = "You are a helpful assistant."
user_msg = "Write a haiku about running Ollama in AI Core"
json_data = {
    "model": model,
    "messages": [
            {
                "role": "system",
                "content": sys_msg
            },
            {
                "role": "user",
                "content": user_msg
            }
    ],
    "stream": False
}

response = requests.post(url=chat_api_endpoint, headers=headers, json=json_data)
print('Result:', response.text)

Result: {"model":"phi3:14b","created_at":"2024-05-23T09:56:57.319052141Z","message":{"role":"assistant","content":"AI's swift strides on data plains,  \n\nOllama core's logic runs free,  \n\nWisdom flows like streams."},"done_reason":"stop","done":true,"total_duration":1647161706,"load_duration":1149212,"prompt_eval_count":29,"prompt_eval_duration":297175000,"eval_count":35,"eval_duration":1263617000}


##### 5.3 Sample#3: Chain of Thought
Now let's conduct a test on a basic sample of Chain of Thought

In [23]:
sys_msg = "You are a helpful assistant."
user_msg = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?Let's thinks step by step."

json_data = {
    "model": model,
    "messages": [
            {
                "role": "system",
                "content": sys_msg
            },
            {
                "role": "user",
                "content": user_msg
            }
    ],
    "stream": False
}

response = requests.post(url=chat_api_endpoint, headers=headers, json=json_data)
print('Result:', response.text)

Result: {"model":"phi3:14b","created_at":"2024-05-23T09:57:15.688146874Z","message":{"role":"assistant","content":"Step 1: Roger starts with 5 tennis balls.\nStep 2: He buys 2 more cans of tennis balls, and each can contains n tennis balls (in this case n=3).\nStep 3: Calculate the total number of tennis balls in the new cans by multiplying the number of cans (2) with the number of tennis balls per can (3): 2 * 3 = 6 tennis balls.\nStep 4: Add the initial number of tennis balls Roger had to the total number of tennis balls from the new cans: 5 + 6 = 11 tennis balls.\n\nSo, Roger now has a total of 11 tennis balls."},"done_reason":"stop","done":true,"total_duration":6079293933,"load_duration":2207541,"prompt_eval_count":50,"prompt_eval_duration":338494000,"eval_count":152,"eval_duration":5647370000}


##### 5.4 Sample#4: Customer Message Processing with OpenAI-compatible Chat Completion API
In our sample [btp-industry-use-cases/04-customer-interaction-gpt4](https://github.com/SAP-samples/btp-industry-use-cases/tree/main/04-customer-interaction-gpt4),GPT-3.5/4 is used to process customer messages in customer interactions and output in json schema with plain prompting.
- Summarize customer message into title and a short description
- Analyze the sentiment of the customer message
- Extract the entities from the customer message, such as customer, product, order no etc.

Let's see if the same scenario could be achieved with mistral-7b.


In [46]:
# Let's test its openai-compatible chat completion api with to process customer message with
# summarization, sentiment analysis and entities extraction and output as json
sys_msg = '''
You are an AI assistant to process the input text. Here are your tasks on the text.
1.Apply Sentiment Analysis
2.Generate a title less than 100 characters,and summarize the text into a short description less than 200 characters
3.Extract the entities such as customer,product,order,delivery,invoice etc from the text Here is a preliminary list of the target entity fields and description. Please extract all the identifiable entities even not in the list below. Don't include any field with unknown value. 
-customer_no: alias customer number, customer id, account id, account number which could be used to identify a customer.
-customer_name: customer name, account name
-customer_phone: customer contact number. -product_no: product number, product id
-product_name
-order_no: sales order number, order id
-order_date 
-delivery_no: delivery number, delivery id
-delivery_date: delivery date, shipping date
-invoice_no: alias invoice number, invoice id, receipt number, receipt id etc. which can be used to locate a invoice.
-invoice_date: invoice date, purchase date
-store_name
-store_location
etc.
    
For those fields not in list must follow the Snakecase name conversation like product_name, no space allow. 

Output expected in JSON format as below: 
{\"sentiment\":\"{{Positive/Neutral/Negative}}\",\"title\":\"{{The generated title based on the input text less than 100 characters}}\",\"summary\":\"{{The generated summary based on the input text less than 300 characters}}\",\"entities\":[{\"field\":\"{{the extracted fields such as product_name listed above}}\",\"value\":\"{{the extracted value of the field}}\"}]}
'''

user_msg = '''
Input text: 
Everything was working fine one day I went to make a shot of coffee it stopped brewing after 3 seconds Then I tried the milk frother it stopped after 3 seconds again I took it back they fixed it under warranty but it’s happening again I don’t see this machine lasting more then 2 years to be honest I’m spewing I actually really like the machine It’s almost like it’s losing pressure somewhere, they wouldn’t tell my what the problem was when they fixed it.. Purchased at Harvey Norman for $1,349. 
Product is used: Several times a week
 
JSON:
'''

json_data = { 
  "model": model,
  "response_format": {"type": "json_object"}, #JSON mode
  "messages": [
            {
                "role": "system",
                "content": sys_msg
            },
            {
                "role": "user",
                "content": user_msg
            }
        ]
}

response = requests.post(url=openai_chat_api_endpoint, headers=headers, json=json_data)
print('Result:', response.text)

Result: {"id":"chatcmpl-518","object":"chat.completion","created":1716559993,"model":"phi3:14b","system_fingerprint":"fp_ollama","choices":[{"index":0,"message":{"role":"assistant","content":"{\n    \"sentiment\": \"Negative\",\n    \"title\": \"Coffee Machine Issues and Warranty Repair\",\n    \"summary\": \"User experiences issues with coffee machine pressure loss. Has been repaired under warranty but problems persist. Purchased at Harvey Norman.\",\n    \"entities\": [\n        {\"field\": \"customer_name\", \"value\": null},\n        {\"field\": \"product_name\", \"value\": \"Coffee Machine\"},\n        {\"field\": \"store_name\", \"value\": \"Harvey Norman\"}\n    ]\n}"},"finish_reason":"stop"}],"usage":{"prompt_tokens":575,"completion_tokens":133,"total_tokens":708}}



##### 5.5 Sample#5: Citizen Reporting App for Public Administrations with OpenAI-compatible Chat Completion API
In our sample [btp-generative-ai-hub-use-cases/01-social-media-citizen-reporting-genai-hub](https://github.com/SAP-samples/btp-generative-ai-hub-use-cases/tree/main/01-social-media-citizen-reporting-genai-hub-genai-hub), the use case is around a fictitious city called "Sagenai City" facing challenges in managing and tracking maintenance in public areas. The city wants to improve the way they handle reported issues from the citizens, by analyzing social media posts & making informed decisions and so effectively tracking & managing issues in public spaces. and output in json schema with plain prompting.
- Category
- Priority
- Summary
- Description
- Address
- Sentiment

Let's see if the same scenario could be achieved with open-source llm.


In [45]:
sys_msg = "You're a helpful assistant"
user_msg = '''
Extract information from the following social media post: 
redditPostId: 198qqqm, 
author: jacobtan89, 
title: Dirty public area, 
message: The public area on Oakwood Road in Sagenai is in a disgraceful state with piles of rubbish and litter scattered everywhere. The author is frustrated with the local authorities for not maintaining cleanliness despite the taxes they pay. They hope for immediate action. #CleanUpYourAct #OakwoodRoadNightmare #DisgustingNeighborhood 
Coordinates:(51.57470453612761,0.003792117010085437), 
postingDate: 2024-01-17T07:13:48.000Z            

The output should be a only json code snippet formatted in the following schema
{
    "address": string  // Extract the address where the issue has been noticed. Return the street only and omit the town or country. For example: Oakwood Road.
    "category": string  // Identify if the social media reports a situation related to one of the following categories:             
        1. PUBLIC CLEANLINESS: Dirty public areas, overflowing dustbins and littering. Bulky waste in common areas.  
        2. ROADS & FOOTPATHS: Including covered linkways, signboards & streetlights. E.g. Pot holes, huge cracks, etc.
        3. FACILITY & PARK MAINTENANCE: Fallen trees, overgrown grass, and maintenance of park lighting and facilities.
        4. PESTS: Sighting of bees and hornets, potential mosquito breeding sites, and more.
        5. DRAINS & SEWERS: Choked, overflowing, or damaged drains, bad sewage smells, flooding.   
        Output the category name. If none of the categories fits, or in doubt, return OTHER - PLEASE CHECK.  
    "description": string  // Summarize the issue that is being reported in not more that 300 characters and a neutral tone.
    "location": string  // Extract the coordinates where the issue has been notices. The format should be: (51.57470453612761,0.003792117010085437).
    "priority": string  // Identify the priority to be given to the reported issues:
        4-Low : the issue does not pose any problem with public safety and does not necessarily need to be handled urgently. 
        3-Medium : the issue does not cause any immediate danger, but it has significant and negative impact on the daily life of people in the neighborhood.
        2-High : the issue needs to be resolved quickly because it can potentially cause dangerous situations or disruptions. 
        1-Very High : the issue needs to be handled as soon as possible, as it is a matter of public safety. Return the priority level. If in doubt, return 3-Medium    
    "sentiment": string  // Extract the sentiment of the post: 
        1. NEUTRAL: if the issue is reported politely
        2. NEGATIVE: if the post shows irritation, impatience, annoyance
        3. VERY NEGATIVE: the post expresses rage, hatred
    "summary": string  // Summarize the issue that is being reported in 40 characters and a neutral tone.
}

JSON:
'''

json_data = {
  "model": model,
  "response_format": {"type": "json_object"}, #JSON mode
  "messages": [
            {
                "role": "system",
                "content": sys_msg
            },
            {
                "role": "user",
                "content": user_msg
            }
        ]
}

response = requests.post(url=openai_chat_api_endpoint, headers=headers, json=json_data)
print('Result:', response.text)

Result: {"id":"chatcmpl-575","object":"chat.completion","created":1716559693,"model":"phi3:14b","system_fingerprint":"fp_ollama","choices":[{"index":0,"message":{"role":"assistant","content":"{\n    \"address\": \"Oakwood Road\",\n    \"category\": \"PUBLIC CLEANLINESS\",\n    \"description\": \"Piles of rubb. litter scattered everywhere on Oakwood Rd, Sagenai. Local govt not cleaning despite taxes.\",\n    \"location\": \"(51.57470453612761, 0.003792117지)\",\n    \"priority\": \"4-Low\",\n    \"sentiment\": \"NEUTRAL\",\n    \"summary\": \"Oakwood Road littered and dirty\"\n}"},"finish_reason":"stop"}],"usage":{"prompt_tokens":721,"completion_tokens":146,"total_tokens":867}}

